# scraping한 rawdata 전처리 - review
- 형식에 맞게 클리닝

In [1]:
import pandas as pd

## review 데이터 클리닝 (진행 중)
1. review 데이터 합치기
2. 클리닝
3. 토큰화

### 1. review 데이터 합치기

In [2]:
import os

def concat_review_data():
    folder_path = "../data/rawdata/"
    infos = []

    for file_path in os.listdir(folder_path):
        if "review" in file_path:
            info = pd.read_csv(folder_path + file_path, converters={"performance_id" : str})
            print(f"[{file_path}]")
            print(info.info())
            infos.append(info)

    df = pd.concat(infos, ignore_index=True)
    df.to_csv(f"../data/preprocessed_data/total_review.csv", index=False, encoding='utf-8')

    return df

In [3]:
# review = concat_review_data()

In [4]:
review = pd.read_csv("../data/preprocessed_data/total_review.csv")

In [5]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38655 entries, 0 to 38654
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  38655 non-null  object
 1   review_id       38655 non-null  int64 
 2   rating          38655 non-null  int64 
 3   user_id         38655 non-null  object
 4   date            38655 non-null  object
 5   view_count      38655 non-null  object
 6   like_count      38655 non-null  object
 7   title           38642 non-null  object
 8   text            38654 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.7+ MB


In [6]:
review.head()

,performance_id,review_id,rating,user_id,date,view_count,like_count,title,text
0,23008837,0,5,tmdfl0***예매자,2024.02.05,조회 6,공감 0\n공감하기,추천합니다,최고의극
1,23008837,1,5,youi1***예매자,2024.01.02,조회 22,공감 0\n공감하기,레베카 공연 최고!!!!!!,배우들의 연기와 감정 최고였음!!!! 앙코르 기념공연 기대되요!!!
2,23008837,2,5,wani***예매자,2023.12.18,조회 13,공감 0\n공감하기,역시 레베카다,최애 공연입니다. 볼때마다 레전드 갱신!
3,23008837,3,5,tsk***예매자,2023.12.16,조회 48,공감 0\n공감하기,옥주연님의 덴버스 부인을 보고,5년 전? 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...
4,23008837,4,5,kys95***예매자,2023.12.13,조회 20,공감 0\n공감하기,믿고 보는 옥댄버!!!,물개박수 치면서 관람 했어요!!!\n정말 인생 뮤지컬이었던:)\n옥댄버 성량이 블루...


### 2. 클리닝
- column별 type 맞추기
- user_id: id / 예매자 여부 분할
- view_count: 조회 없애기 (숫자만 남기기)
- like_count: 공감 ~ \n공감하기 없애기 (숫자만 남기기)
- 베스트 리뷰 삭제 (중복임)
- 라이브 리뷰라는 표시 삭제

In [7]:
# 조회수 숫자형으로
review["view_count"] = review["view_count"].apply(lambda x: int(x.split(" ")[1]))

# 공감수 숫자형으로
review["like_count"] = review["like_count"].apply(lambda x: int(x[3:-5]))

In [8]:
# id에 나타나는 예매자 여부 삭제
review['user_id'] = review['user_id'].str.split("\*\*\*", expand=True)[0] + "***"

- 베스트 리뷰 제거

In [9]:
# 베스트 리뷰로 등록되지 않았음에도 중복 리뷰가 있을 경우를 위해 title까지 포함해 1차 중복 제거 진행
# title 제외(베스트 키워드가 포함되므로)한 뒤 중복 제거 베스트 리뷰가 무조건 빠르므로 keep="last"
review.drop_duplicates(subset=["performance_id", "user_id", "date", "view_count", "like_count", "title", "text"], keep='first', inplace=True, ignore_index=True)
review.drop_duplicates(subset=["performance_id", "user_id", "date", "view_count", "like_count", "text"], keep='last', inplace=True, ignore_index=True)

In [10]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24854 entries, 0 to 24853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24854 non-null  object
 1   review_id       24854 non-null  int64 
 2   rating          24854 non-null  int64 
 3   user_id         24854 non-null  object
 4   date            24854 non-null  object
 5   view_count      24854 non-null  int64 
 6   like_count      24854 non-null  int64 
 7   title           24841 non-null  object
 8   text            24853 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


- title, text Null(결측치) 값 ''으로 치환
    - title, text에만 결측치 존재 -> df.fillna('')

In [11]:
# Null 값 확인
review[review["title"].isnull() | review["text"].isnull()]

,performance_id,review_id,rating,user_id,date,view_count,like_count,title,text
5799,24001224,647,5,wjdfksdl3***,2024.04.08,1,0,NaN,초연때 처음 봤던 캐스팅으로 다시 봤는데 정말 너무너무너무너무 좋았어요ㅠㅠ 오즈가 ...
7766,24001224,2685,5,wjdfksdl3***,2024.03.04,1,0,NaN,다시 돌아온 오즈!! 넘버들도 너무 좋고 너무 재밌어요ㅠㅠ 이번 시즌은 꼭 OST가...
8226,24002031,313,5,wjdfksdl3***,2024.04.15,0,0,NaN,항상 페어마다 다른 재미가 있어서 너무 좋아요~
8256,24002031,345,5,wjdfksdl3***,2024.04.15,1,0,NaN,웨스턴 스토리 넘버가 너무 좋아요bb 이번 시즌 전캐전곡 OST 꼭 나왔으면 좋겠습...
8765,24002031,866,5,wjdfksdl3***,2024.04.08,1,0,NaN,페어마다 다른 매력을 가지고 있어서 볼 때마다 새로운 공연을 보는 것 같아요! 너무...
8784,24002031,886,5,wjdfksdl3***,2024.04.08,0,0,NaN,"아 정말 웃다가 울고 나왔어요ㅠㅠㅋㅋㅋㅋㅋㅋㅋ 와이어트의 화려한 골반 너무 잘봤고,..."
9291,24002031,1421,5,wjdfksdl3***,2024.04.01,1,0,NaN,너무 웃겨서 웃다가 눈물났어요ㅋㅋㅋㅋㅋㅋ 찐찐이와 원종''환'' 정말 최고!!!
9737,24002031,1887,5,wjdfksdl3***,2024.03.25,2,0,NaN,와이어트의 골반춤이 자꾸 떠올라요ㅠㅋㅋㅋㅋ 같이 골반을 돌려야할 것 같은데.. OS...
9785,24002031,1937,5,wjdfksdl3***,2024.03.25,2,0,NaN,웃다가 배꼽 빠질뻔했어요ㅋㅋㅋㅋㅋㅋㅋ 배우분들 다 너무 노래도 잘하시고 웃기시고 최고bb
10198,24002031,2358,5,wjdfksdl3***,2024.03.18,4,0,NaN,어떤 캐스팅으로 봐도 너무 재밌는 웨스턴 스토리! 이번 시즌엔 꼭 전캐전곡 OST가...


In [12]:
# title, text 결측치 채우기
# '' 로 진행하면 csv로 저장 후 다시 불러왔을 때 Null로 표시되기 때문에 ' '로 진행
review.fillna(' ', inplace=True)

In [13]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24854 entries, 0 to 24853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24854 non-null  object
 1   review_id       24854 non-null  int64 
 2   rating          24854 non-null  int64 
 3   user_id         24854 non-null  object
 4   date            24854 non-null  object
 5   view_count      24854 non-null  int64 
 6   like_count      24854 non-null  int64 
 7   title           24854 non-null  object
 8   text            24854 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


- title 말미에 붙는 "\n라이브 후기" 제거
    - title에 라이브 후기가 아님에도 \n이 있는 경우는 없다
    - 제목이 없는데 라이브 후기인 경우 -> "\n라이브 후기" -> "" 형태가 되도록

In [14]:
review["title"] = review["title"].apply(lambda x : x.split("\n")[0]).replace("라이브 후기", "")

- column 이름 (field 이름) 통일

In [15]:
review.columns

Index(['performance_id', 'review_id', 'rating', 'user_id', 'date',
       'view_count', 'like_count', 'title', 'text'],
      dtype='object')

In [16]:
review.rename(columns={"review_id" : "review_num", "text" : "content"}, inplace=True)

- 불필요한 리뷰 삭제
    - 가장 긴 content 길이를 가진 두 가지 리뷰는 같은 단어를 반복해 작성한 결과였다 -> 삭제 ㅓ리
    - index: 7288, performance_id = 24001224, review_num = 2194
    - index: 18612, performance_id = 19018229#, review_num = 10490

In [17]:
review["content_len"] = [len(content) for content in review["content"]]

idx = review.sort_values(by="content_len")[-2:].index
review.drop(idx, inplace=True)

review.drop("content_len", axis=1, inplace=True)

In [18]:
review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24852 entries, 0 to 24853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24852 non-null  object
 1   review_num      24852 non-null  int64 
 2   rating          24852 non-null  int64 
 3   user_id         24852 non-null  object
 4   date            24852 non-null  object
 5   view_count      24852 non-null  int64 
 6   like_count      24852 non-null  int64 
 7   title           24852 non-null  object
 8   content         24852 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.9+ MB


In [19]:
review.to_csv("../data/preprocessed_data/total_review_cleaning.csv", index=False, encoding='utf-8')

---

## 토큰화
- 특수문자 제거 (한글만 남기기)
- 맞춤법 검사
- 형태소 분석 (명사 추출)
- 불용어 제거

- 한글 이외의 문자 제거

In [20]:
review = pd.read_csv("../data/preprocessed_data/total_review_cleaning.csv")

In [21]:
review.head()

,performance_id,review_num,rating,user_id,date,view_count,like_count,title,content
0,23008837,0,5,tmdfl0***,2024.02.05,6,0,추천합니다,최고의극
1,23008837,1,5,youi1***,2024.01.02,22,0,레베카 공연 최고!!!!!!,배우들의 연기와 감정 최고였음!!!! 앙코르 기념공연 기대되요!!!
2,23008837,2,5,wani***,2023.12.18,13,0,역시 레베카다,최애 공연입니다. 볼때마다 레전드 갱신!
3,23008837,3,5,tsk***,2023.12.16,48,0,옥주연님의 덴버스 부인을 보고,5년 전? 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...
4,23008837,4,5,kys95***,2023.12.13,20,0,믿고 보는 옥댄버!!!,물개박수 치면서 관람 했어요!!!\n정말 인생 뮤지컬이었던:)\n옥댄버 성량이 블루...


In [22]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24852 entries, 0 to 24851
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24852 non-null  object
 1   review_num      24852 non-null  int64 
 2   rating          24852 non-null  int64 
 3   user_id         24852 non-null  object
 4   date            24852 non-null  object
 5   view_count      24852 non-null  int64 
 6   like_count      24852 non-null  int64 
 7   title           24850 non-null  object
 8   content         24852 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


- 특수문자 제거 (한글만 남기기)

In [29]:
review['title'] = review['title'].str.replace('[^가-힣]', ' ', regex = True)
review['content'] = review['content'].str.replace('[^가-힣]', ' ', regex = True)

review[['title', 'content']].head()

,title,content
0,추천합니다,최고의극
1,레베카 공연 최고,배우들의 연기와 감정 최고였음 앙코르 기념공연 기대되요
2,역시 레베카다,최애 공연입니다 볼때마다 레전드 갱신
3,옥주연님의 덴버스 부인을 보고,년 전 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...
4,믿고 보는 옥댄버,물개박수 치면서 관람 했어요 정말 인생 뮤지컬이었던 옥댄버 성량이 블루스퀘...


- 맞춤법 검사
    - title은 최대 100자로 hanspell의 제약에 걸리지 않는다
    - context는 최대 11664자가 넘는 경우도 있어 hanspell 사용 시 유의해야 한다
        -> 공백 기준으로 500자씩 끊어 진행 후 ' '.join()

In [24]:
# 500자 이상인 경우 -> 공백 단위로 500자씩 split해 return
# 단어가 잘리지 않도록 공백 단위로 자르도록 함

def split_paragraph_by_words(paragraph, max_length=500):
    words = paragraph.split()  # 공백을 기준으로 단어들을 분리
    
    chunks = []
    current_chunk = ""
    for word in words:
        if len(current_chunk) + len(word) + 1 <= max_length:  # 현재 chunk에 현재 단어를 추가해도 최대 길이를 초과하지 않을 때
            if current_chunk:  # 현재 chunk가 비어있지 않을 때
                current_chunk += " "  # 단어 사이에 공백 추가
            current_chunk += word
        else:
            chunks.append(current_chunk)
            current_chunk = word  # 현재 단어를 새로운 chunk로 시작
    if current_chunk:  # 마지막 chunk 추가
        chunks.append(current_chunk)
    return chunks

In [ ]:
# from hanspell import spell_checker

# review.fillna(' ', inplace=True)

# spelled_texts = []
# i = 0   # 확인용 변수
# for text in review["title"].values:
#     i += 1

#     spelled_text = spell_checker.check(text).checked
#     spelled_texts.append(spelled_text)

#     print(i, text, spelled_text)

# review["title_spelled"] = spelled_texts

In [ ]:
# from hanspell import spell_checker

# spelled_texts = []
# i = 0
# for content in review["content"].values:
#     # 500자 단위, 공백 기준으로 split
#     split_contents = split_paragraph_by_words(content)

#     # 맞춤법 검사 진행
#     split_spellled = []
#     for text in split_contents:
#         spelled_text = spell_checker.check(text).checked
#         split_spellled.append(spelled_text)

#     # 공백 기준으로 join
#     spelled_texts.append(' '.join(split_spellled))
#     print(i, content, ' '.join(split_spellled))
#     i += 1

# review["title_spelled"] = spelled_texts

> content에 대해 hanspell을 실행하면서 JSONDecoder 오류가 계속 진행됨..
>   - `JSONDecodeError: Expecting value: line 1 column 1 (char 0)`
> - 해결되지 않아 일단 맞춤법은 제외하고 진행

- 토큰화 (명사 추출)
    - [형태소 분석기 비교](https://jongsky.tistory.com/31)

In [30]:
from konlpy.tag import Okt

def get_nouns(text):
    okt = Okt()

    nouns = okt.nouns(text)
    #-- 추출된 명사 중에서 길이가 1보다 큰 단어만 추출
    word = [w for w in nouns if len(w) > 1] 
    return word

In [31]:
review.fillna(' ', inplace=True)
review["title_n"] = [get_nouns(title) for title in review["title"].values]

In [32]:
review[["title", "title_n"]]

,title,title_n
0,추천합니다,[추천]
1,레베카 공연 최고,"[레베카, 공연, 최고]"
2,역시 레베카다,"[역시, 레베카]"
3,옥주연님의 덴버스 부인을 보고,"[주연, 버스, 부인, 보고]"
4,믿고 보는 옥댄버,[옥댄버]
...,...,...
24847,갓뼘사이,[사이]
24848,완전 추천요,"[완전, 추천]"
24849,재관람 건너편으로 갔네요,"[관람, 건너편]"
24850,힐링했어요,[힐링]


In [33]:
review["content_n"] = [get_nouns(content) for content in review["content"].values]

In [34]:
review[["content", "content_n"]]

,content,content_n
0,최고의극,[최고]
1,배우들의 연기와 감정 최고였음 앙코르 기념공연 기대되요,"[배우, 연기, 감정, 최고, 앙코르, 기념, 공연]"
2,최애 공연입니다 볼때마다 레전드 갱신,"[최애, 공연, 볼때, 레전드, 갱신]"
3,년 전 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...,"[버스, 부인, 보고, 충격, 주연, 연기, 기대, 커서, 걱정, 정말, 뮤지컬, ..."
4,물개박수 치면서 관람 했어요 정말 인생 뮤지컬이었던 옥댄버 성량이 블루스퀘...,"[물개, 박수, 치면, 관람, 정말, 인생, 뮤지컬, 옥댄버, 성량, 블루, 스퀘어]"
...,...,...
24847,가벼운 마음으로 공연보러왓다가 미친듯이 웃고가네요 한번보고가세요 두번보고가...,"[마음, 공연, 다가, 한번, 세번]"
24848,여자친구랑 봤는데 스토리도 좋구 중간중간 재밌는 요소들도 많아서 정말 웃으면서 잘...,"[여자친구, 스토리, 중간, 중간, 요소, 정말, 추천]"
24849,무대도 더 산뜻해지고 무대가 가까워져서 너무좋았어요 또보러올게요,"[무대, 무대]"
24850,코로나때문에 연극 보고싶어도 못봐서 오늘 정말 기대많이 하고 봤는데 기대이상이에요 ...,"[코로나, 때문, 연극, 오늘, 정말, 이상, 연기자, 연기, 내용, 구성, 시간,..."


In [36]:
review.head()

,performance_id,review_num,rating,user_id,date,view_count,like_count,title,content,title_n,content_n
0,23008837,0,5,tmdfl0***,2024.02.05,6,0,추천합니다,최고의극,[추천],[최고]
1,23008837,1,5,youi1***,2024.01.02,22,0,레베카 공연 최고,배우들의 연기와 감정 최고였음 앙코르 기념공연 기대되요,"[레베카, 공연, 최고]","[배우, 연기, 감정, 최고, 앙코르, 기념, 공연]"
2,23008837,2,5,wani***,2023.12.18,13,0,역시 레베카다,최애 공연입니다 볼때마다 레전드 갱신,"[역시, 레베카]","[최애, 공연, 볼때, 레전드, 갱신]"
3,23008837,3,5,tsk***,2023.12.16,48,0,옥주연님의 덴버스 부인을 보고,년 전 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...,"[주연, 버스, 부인, 보고]","[버스, 부인, 보고, 충격, 주연, 연기, 기대, 커서, 걱정, 정말, 뮤지컬, ..."
4,23008837,4,5,kys95***,2023.12.13,20,0,믿고 보는 옥댄버,물개박수 치면서 관람 했어요 정말 인생 뮤지컬이었던 옥댄버 성량이 블루스퀘...,[옥댄버],"[물개, 박수, 치면, 관람, 정말, 인생, 뮤지컬, 옥댄버, 성량, 블루, 스퀘어]"


In [35]:
review.to_csv("../data/preprocessed_data/total_review_token.csv", index=False, encoding="utf-8")